In [7]:
import requests as req
import pandas as pd
import numpy as np
import json

In [12]:
def fetch_data_json(link,params=[]):
    fetch_link =req.get(link)
    return pd.DataFrame.from_dict(fetch_link.json())


In [11]:
link_15 = 'https://raw.githubusercontent.com/ndejong/philippines-data/master/data/2015%20Population%20Density_web_clean-columns.json'
data_20 = pd.DataFrame(pd.read_csv('ph_cities_20.csv'))
data_15 = fetch_data_json(link_15)

In [16]:
print(len(data_15['city_municipality']))
print(len(data_20['city']))

1634
1007


In [49]:
cities_data = [data_15['city_municipality'],data_20['city']]
cities_new = {'citynew_15':[],'citynew_20':[],'cityset_15':None,'cityset_20':None}
for i,data in enumerate(cities_data):
    for city in data:
        new_city = city.lower().replace('city','').replace('of','').strip()
        cities_new[list(cities_new.keys())[i]].append(new_city)
    cities_new[list(cities_new.keys())[i+2]] = set(cities_new[list(cities_new.keys())[i]])
cities_common=cities_new['cityset_20'].intersection(cities_new['cityset_15'])

In [59]:
data_15_01 = pd.concat((data_15,pd.Series(cities_new['citynew_15'],name='newcity_15')),axis=1)
data_20_01 = pd.concat((data_20,pd.Series(cities_new['citynew_20'],name='newcity_20')),axis=1)

In [110]:
data_15_02 = data_15_01[data_15_01['newcity_15'].isin(list(cities_common))].reset_index()
data_20_02 = data_20_01[data_20_01['newcity_20'].isin(list(cities_common))].reset_index()

In [111]:
new_data = {'lat':[],'lng':[],'population':[],'population_proper':[]}
for i,city in enumerate(data_15_02['newcity_15']):
    data = data_20_02[data_20_02['newcity_20']==city].iloc[0,0:]
    for j,key in enumerate(list(new_data.keys())):
        if j >= 1:
            new_data[key].append(float(data[key]))
        else:
            new_data[key].append(float(data[key].mean()))

In [116]:
comb_data = pd.concat((data_15_02,pd.DataFrame.from_dict(new_data)),axis=1)
comb_data.columns

Index(['index', 'region', 'region_name', 'province', 'city_municipality',
       'is_capital', 'population_2015', 'land_area_sqkm', 'population_density',
       'newcity_15', 'lat', 'lng', 'population', 'population_proper'],
      dtype='object')

In [118]:
comb_data.columns = ['index','region','region_name','province', 'city_municipality',
                     'is_capital', 'population_2015', 'land_area_sqkm', 'population_density_2015',
                    'cityid_name','latitude','longitude','population_recent','population_proper_recent']

In [120]:
comb_data.to_csv('ph_cities_1520.csv')